In [ ]:
import requests
from tqdm import tqdm

url = "https://data.caida.org/datasets/security/ddos-20070804/to-victim/ddostrace.to-victim.20070804_144936.pcap.gz"
username = "ajayprasad0008@gmail.com"
password = "paranomal@aT"


response = requests.get(url, auth=(username, password), stream=True)

if response.status_code == 200:
    file_size = int(response.headers.get('content-length', 0))
    chunk_size = 1024  # 1 KB
    with open("ddostrace.pcap.gz", "wb") as f, tqdm(
        desc="Downloading",
        total=file_size,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=chunk_size):
            bar.update(len(data))
            f.write(data)
    print("File downloaded successfully.")
else:
    print("Failed to download the file. Check your username and password.")


Downloading: 100%|██████████| 771M/771M [02:22<00:00, 5.67MB/s]


File downloaded successfully.


File Downloaded

In [ ]:
import gzip
import shutil

input_file_path = "ddostrace.pcap.gz"
output_file_path = "ddostrace.pcap"

with gzip.open(input_file_path, 'rb') as f_in:
    with open(output_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("File extracted successfully.")


File extracted successfully.


Extracted

In [ ]:
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444328 sha256=cbbb9170812129057a7ded25965cfd0c811e3a656df85e9d3f3a4568d170a6b4
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
import csv
from scapy.all import rdpcap

def pcap_to_csv(input_pcap, output_csv):
    packets = rdpcap(input_pcap)

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'Timestamp', 'Source IP', 'Destination IP', 'Source MAC', 'Destination MAC',
            'Protocols', 'Packet Length', 'Port', 'IP Protocol', 'Inter-Arrival Time',
            'Payload Data'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for packet in packets:
            timestamp = packet.time
            src_ip = packet.getlayer('IP').src if 'IP' in packet else 'N/A'
            dst_ip = packet.getlayer('IP').dst if 'IP' in packet else 'N/A'
            src_mac = packet.getlayer('Ether').src if 'Ether' in packet else 'N/A'
            dst_mac = packet.getlayer('Ether').dst if 'Ether' in packet else 'N/A'
            protocols = packet.getlayer('IP').payload.name if 'IP' in packet else 'N/A'
            packet_len = len(packet)
            port = packet.getlayer('TCP').dport if 'TCP' in packet else (packet.getlayer('UDP').dport if 'UDP' in packet else 'N/A')
            ip_protocol = packet.getlayer('IP').proto if 'IP' in packet else 'N/A'
            inter_arrival_time = packet.time - timestamp if timestamp else 'N/A'
            payload_data = str(packet.payload) if hasattr(packet, 'payload') else 'N/A'

            writer.writerow({
                'Timestamp': timestamp,
                'Source IP': src_ip,
                'Destination IP': dst_ip,
                'Source MAC': src_mac,
                'Destination MAC': dst_mac,
                'Protocols': protocols,
                'Packet Length': packet_len,
                'Port': port,
                'IP Protocol': ip_protocol,
                'Inter-Arrival Time': inter_arrival_time,
                'Payload Data': payload_data
            })

if __name__ == "__main__":
    input_pcap_file = "ddostrace.pcap"  # Replace with your input pcap file
    output_csv_file = "output.csv"    # Replace with your desired output csv file

    pcap_to_csv(input_pcap_file, output_csv_file)
